# PART 3: Modeling and stress-testing

### Overview

The primary goal of our backtesting is to assess the performance and robustness of the DivProtocol strategy in comparison to the SimpleUniV2Strategy, particularly for the USTC/BUSD market pair.

Model Parameters and Assumptions:

- **Data Source:** Historical trade data is sourced from Binance for the market pair USTC/BUSD.

- **Date Range:** The data spans from January 1, 2023, to June 30, 2023.

- **Liquidity:** For both strategies, we assumed an initial liquidity of $1,000,000 in the USTC/BUSD market.

- **Trade Execution:** The Constant Product Automated Market Maker (AMM) mechanism, a popular method in decentralized finance, was employed to simulate trade execution.

- **Performance Visualization:** Using the Bokeh library, we visualized the performance of both strategies over the specified time range, offering a clear depiction of their respective strengths and weaknesses.


In [1]:
import os
import numpy as np
import pandas as pd
from IPython.display import display
from IPython.core.display import HTML
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Div
from bokeh.layouts import row

output_notebook()
pd.options.display.float_format = '{:,.5f}'.format
HTML('<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">')
# HTML('<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css">')

Loading BokehJS ...

In [2]:
from binance import Client
from terra_algo_backtest.utils import format_df
from terra_algo_backtest.plotting import new_trade_figure, plot_price_ratio
from terra_algo_backtest.binance_loader import new_binance_client

# Replace these with your Binance API key and secret
client = new_binance_client(os.getenv("BINANCE_API_KEY"), os.getenv("BINANCE_API_SECRET"))

mkt_pair = 'USTC/BUSD'
frequency = Client.KLINE_INTERVAL_1HOUR
start, end = '2023-01-01 00:00:00', '2023-06-30 23:59:59'
qty_factor = 0.01 # pct of binance historical volume

df_trades = client.get_trade_data(mkt_pair, frequency, start, end)
df_trades["quantity"] = qty_factor*df_trades["quantity"]
show(Div(text=format_df(df_trades.head(10), width=600)))

## Strategies

While our current analysis provides insights into strategy performance under historical conditions, it's essential to stress-test strategies under extreme market conditions to ensure robustness. This would involve simulating scenarios where multiple market factors go awry simultaneously, helping us gauge the strategy's resilience.

This section provides a comprehensive breakdown of the backtesting approach, model parameters, and initial results. Further iterations would benefit from more exhaustive stress-testing and possibly the exploration of other strategies or market pairs.

### Risks and Considerations

- **Dependence on Historical Data:** Like all backtesting approaches, our results are based on past market conditions. Future performance can vary based on a multitude of unpredictable factors.  

- **Liquidity Concerns:** While we assumed a fixed initial liquidity, real-world scenarios can see varying liquidity levels, which might impact strategy performance.

- **Strategy Complexity:** The DivProtocolStrategy is more intricate than the SimpleUniV2Strategy. While this can offer more nuanced trading decisions, it also means there are more parameters and conditions to monitor and adjust.

### SimpleUniV2Strategy 

The SimpleUniV2Strategy integrates a CP-AMM for its trading activities. The strategy optionally includes an arbitrage mechanism that, when enabled, seeks and executes potential arbitrage opportunities. This allows to keep the mid price of the pools in line with the market price.

1. **Arbitrage Mechanism**: 
   - Arbitrage in this context refers to the practice of taking advantage of a price difference between two or more markets. For instance, if there's a discrepancy between the market price and the CP-AMM's price, an arbitrage trade can be executed to profit from this difference.

2. **Trade Execution Based on Quantity**:
   - Trade orders are then passed to the CP-AMM for execution. The CP-AMM handles the trade, ensuring that it adheres to the constant product formula, and returns the information about the execution.

3. **Strategy Outcome**:
   - The main outcome of the strategy's execution is a list of trade execution information. This list contains details of all trades executed during the strategy's operation, including any arbitrage trades and regular trades based on the simulation data's quantity.
   - This detailed trade information can be used for further analysis, reporting, and to understand the strategy's performance over time.

In [3]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.plot_layout import div_layout
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy, DivProtocolStrategy, DivProtocolParams

liquidity_usd = 1_000_000
# USTC/BUSD market price
base = MarketQuote("USTC/BUSD", df_trades.price.iloc[0])
# BUSD/USD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 1,000,000 USD market for USTC/BUSD with 0.3% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.001)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# div protocol with buy back strategy on top the amm strat
strategy=SimpleUniV2Strategy(cp_amm, arb_enabled=True)
# run simulation
simul = swap_simulation(df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul))

Function 'trade_summary' executed in 0.0078s
Function 'sim_results' executed in 0.0972s
Function 'swap_simulation' executed in 0.4725s


### Observations

#### PnL Breakdown:

**1. Blue Line (Percentage P&L):** 
   - **What is it?** Represents your Profit & Loss (P&L) relative to the initial investment.
   - **Interpretation:** If this line shows +1%, it indicates you've earned 1% on top of the initial liquidity you provided, which is 1M USD. This line gives a direct visual of how your investment is performing over time.

**2. Dashed Green Line (Transaction Fees):**
   - **What is it?** This line shows the transaction fees accumulated over time.
   - **Interpretation:** Consider this as the maximum potential profit you can achieve. It acts as the ceiling for your earnings, purely from fees.

**3. Dashed Red Line (Impermanent Loss):**
   - **What is it?** Represents the losses incurred due to price differences exploited by arbitragers.
   - **Interpretation:** This is the minimum boundary of your P&L. Any decrease in value due to market dynamics and arbitrage activities will approach this line.

---

#### Mid Vs Exec Price:

**What is it?** This measures the difference between the mid price of the liquidity pool and the price at which trades are executed.

**Interpretation:** A consistent discrepancy between these two can suggest frequent arbitrage opportunities or inefficiencies in the market.

---

#### Volume (Quote):

**What is it?** The total trading volume in the past 24 hours, denominated in the quote currency (e.g., BUSD).

**Interpretation:** It provides a sense of market activity. High volumes can indicate significant market events or high liquidity.

---

#### Price Impact:

**What is it?** The change in price caused by trade orders relative to the overall market price.

**Interpretation:** High price impact can reduce profit margins, especially during significant market movements. It's essential to monitor this to understand the efficiency of trades and the liquidity pool's depth.

---

#### Arb Profit:

**What is it?** The profit derived specifically from arbitrage trades. These trades capitalize on price discrepancies between the liquidity pool and the broader market.

**Interpretation:** The first significant spike correlates with a major market drop. Notably, while this arbitrage action generated the highest daily volume, it didn't yield the maximum P&L. This discrepancy is due to the significant price impact, as visualized in the left chart.

---

#### IL Theoretical Vs Actual:

**What is it?** A comparison between the calculated Profit & Loss from actual trades and the theoretical Impermanent Loss (IL) using a predetermined formula.

**Interpretation:** Both values should align closely. If they do, it validates the accuracy of the IL calculation and ensures that the pool's performance is as expected. If not, it could indicate discrepancies in trade executions or inaccuracies in the IL formula.

### DivProtocolStrategy 

The DivProtocolStrategy wraps the SimpleUniV2 strategy, enhancing it with dividend protocol parameters. It maintains a reserve of base and quote assets. When executing trades, it first processes the underlying strategy's trades, applying a divergence tax calculation. Additionally, if the market price is below a designated peg price and there's a positive reserve, buy-back trades are executed, potentially stabilizing the market.

1. **Buy-Back Condition**: The strategy continuously checks if the mid price of the pool is below the specified `peg_price`. If this condition is met and there's a positive reserve, a buy-back trade is considered.

2. **Reserve**: The strategy maintains a reserve that accumulates over time due to the dividend tax mechanism. This reserve can be in the form of base and/or quote assets. 

3. **Buy-Back Trade Calculation**: If the buy-back conditions are met, the strategy calculates the amount of the buy-back trade. This function determines the amount of the asset to buy back, how much of the reserve to use, and if there's a need to borrow any additional assets.

4. **Buy-Back Execution**: If a buy-back trade is determined, the strategy proceeds to execute the buy-back against the pool.

5. **Stabilization Objective**: The main objective behind the buy-back mechanism is to provide price stability. By buying back assets when the price is below the peg, the strategy exerts upward pressure on the price, helping it move closer to the peg price. This action can help reduce price volatility and maintain the asset's price closer to its intended peg.

In summary, the buy-back mechanism in the `DivProtocolStrategy` acts as a stabilizing force. By utilizing the accumulated reserve to purchase assets when their price is below the peg, the strategy aims to mitigate drastic price drops and maintain the asset's value closer to its intended peg price.


In [4]:
from terra_algo_backtest.market import MarketQuote, new_market
from terra_algo_backtest.simulation import swap_simulation
from terra_algo_backtest.plotting import new_simulation_figure
from terra_algo_backtest.plot_layout import div_layout
from terra_algo_backtest.exec_engine import ConstantProductEngine
from terra_algo_backtest.strategy import SimpleUniV2Strategy, DivProtocolStrategy, DivProtocolParams

liquidity_usd = 1_000_000
# USTC/BUSD market price
base = MarketQuote("USTC/BUSD", df_trades.price.iloc[0])
# BUSD/USD market price
quote = MarketQuote("BUSD/USD", 1)
# create a 1,000,000 USD market for USTC/BUSD with 0.3% swap fee 
mkt = new_market(liquidity_usd, quote, base, 0.001)
# create a cp swap execution engine
cp_amm = ConstantProductEngine(mkt)
# div protocol with buy back strategy on top the amm strat
strategy = DivProtocolStrategy(
    strategy=SimpleUniV2Strategy(cp_amm, arb_enabled=True),
    strat_params=DivProtocolParams(peg_price=0.030),
)
# run simulation
simul_div = swap_simulation(df_trades, strategy)
# display results
show(new_simulation_figure(mkt, simul_div, sim_layout_fn=div_layout))

Function 'trade_summary' executed in 0.0068s
Function 'sim_results' executed in 0.1568s
Function 'swap_simulation' executed in 0.8148s


### Observations

#### Price Div Tax ON/OFF

**What is it?** Market price versus price with buyback using proceeds from the divergence protocol   

**Interpretation:** 

- We used a peg at 0.03USD and you can see that with the divergence protocol and buy back the price is either at peg or higher. 

- At the begining as we havent taxed and built a reserve, the price is below peg but once the reserve allows it price is pushed towards the peg 

- At the end, we've exhausted the reserve and we can't buy back enough to keep the peg so the price drops back towards market price


---

#### Div Tax:

**What is it?** Tax Amount collected per day 

**Interpretation:** towards the end as the market price keeps dropping and we keep buying back, we also create more arbitrage opportunity versus market price which in turns results into more tax collected. Like a feedback loop effect

---

#### Volume Div Tax ON/OFF:

**What is it?** The total trading volume in the past 24 hours, denominated in the quote currency (e.g., BUSD). And the volume minus the divergence tax

**Interpretation:** 

---

#### Reserve Base/Quote:

**What is it?**: The reserve account level. Quote is what has been taxed and based (converted to quote) is the USTC accumulated from buy back 

**Interpretation:** 

---

#### Arb Profit:

**What is it?** The profit derived specifically from arbitrage trades. These trades capitalize on price discrepancies between the liquidity pool and the broader market.

**Interpretation:** The first significant spike correlates with a major market drop. Notably, while this arbitrage action generated the highest daily volume, it didn't yield the maximum P&L. This discrepancy is due to the significant price impact, as visualized in the left chart.

---

#### Buy back volume:

**What is it?** A comparison between the calculated Profit & Loss from actual trades and the theoretical Impermanent Loss (IL) using a predetermined formula.

**Interpretation:** Both values should align closely. If they do, it validates the accuracy of the IL calculation and ensures that the pool's performance is as expected. If not, it could indicate discrepancies in trade executions or inaccuracies in the IL formula.

---

#### Price Impact:

**What is it?** The change in price caused by trade orders relative to the overall market price.

**Interpretation:** High price impact can reduce profit margins, especially during significant market movements. It's essential to monitor this to understand the efficiency of trades and the liquidity pool's depth.

---

#### Div Tax (%):

**What is it?** A comparison between the calculated Profit & Loss from actual trades and the theoretical Impermanent Loss (IL) using a predetermined formula.

**Interpretation:** Both values should align closely. If they do, it validates the accuracy of the IL calculation and ensures that the pool's performance is as expected. If not, it could indicate discrepancies in trade executions or inaccuracies in the IL formula.


### Strategy Comparison 

A side-by-side comparison between the two strategies was executed, highlighting their differences in performance metrics over the testing period.

In [5]:
import pandas as pd

def dataframe_difference(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame containing the difference between two DataFrames.

    Args:
    - df1: First DataFrame.
    - df2: Second DataFrame.

    Returns:
    - pd.DataFrame: DataFrame containing the difference.
    """

    # Columns present in both dataframes
    common_columns = df1.columns.intersection(df2.columns)

    # Columns unique to each dataframe
    df1_unique_columns = df1.columns.difference(df2.columns)
    df2_unique_columns = df2.columns.difference(df1.columns)

    # Calculate difference for common columns
    df_diff = df1[common_columns].sub(df2[common_columns].reindex_like(df1), fill_value=0)

    # Add columns unique to df1
    df_diff[df1_unique_columns] = df1[df1_unique_columns]

    # Add columns unique to df2
    df_diff[df2_unique_columns] = -df2[df2_unique_columns]

    return df_diff

df1 = simul_div["breakdown"]
df2 = simul["breakdown"]

# diff_df = dataframe_difference(df1, df2)
simul_diff = {
    "headline": dataframe_difference(simul_div["headline"], simul["headline"]),
    "breakdown": dataframe_difference(simul_div["breakdown"], simul["breakdown"]),
}
show(new_simulation_figure(mkt, simul_diff))
